In [89]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [90]:
def pat_match(patten,saying):
    if is_variable(patten[0]):
        print (patten[0]) 
        return True
    
    else:
        if patten[0] != saying[0]: return True
        else:
            return pat_match(patten[1:],saying[1:])

In [91]:
pat_match("I want to learn ?X ".split(),"I want to learn NLP".split())

?X


True

In [92]:
pat_match ("It is a ?X".split(),"It is a dog".split())

?X


True

# 修改优化，获得每个variable对应的是什么值

In [93]:
def pat_match(pattern,saying):
    if is_variable(pattern[0]):
        return pattern[0],saying[0]
    else:
        if pattern[0] !=saying[0]:return False
        else:
            return pat_match(pattern[1:],saying[1:])

In [94]:
pat_match("I want to have ?X".split(),"I want to have house".split())

('?X', 'house')

In [95]:
pat_match("?X for ?X".split(),"AI  for NLP".split())

('?X', 'AI')

# 进一步优化,匹配多个变量

In [96]:
def pat_match(pattern,saying):
    if not  pattern or not  saying:return []
    
    if is_variable(pattern[0]):
        print(pattern[0])
        return [(pattern[0],saying[0])] + pat_match(pattern[1:],saying[1:])
    
    else:
        if pattern[0] != saying[0]:return []
        else:
            return pat_match(pattern[1:],saying[1:])

In [97]:
pat_match("?X greater ?W than ?Y ?Z".split(),"3 greater 4 than 2".split())

?X
?W
?Y


[('?X', '3'), ('?W', '4'), ('?Y', '2')]

In [98]:
def pat_to_dic(patterns):
    return {k:v for k,v in patterns}

In [99]:
def subsititle(rule,parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0],rule[0])] +subsititle(rule[1:],parsed_rules)

In [100]:
got_patterns =pat_match("I want to buy ?X".split()," I want to buy iPhone".split())

?X


In [101]:
got_patterns

[('?X', 'iPhone')]

In [102]:
subsititle("what if you mean if you got a ?X".split(),pat_to_dic(got_patterns))

['what', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [103]:
' '.join(subsititle("what if you mean if you got a ?X".split(),pat_to_dic(got_patterns)))

'what if you mean if you got a iPhone'

In [104]:
pat_to_dic(got_patterns)

{'?X': 'iPhone'}

In [105]:
join_pat = pat_match("?p need ?X".split(),"iPhone need set".split())

?p
?X


In [106]:
join_pat

[('?p', 'iPhone'), ('?X', 'set')]

In [107]:
subsititle("why does ?p need ?X ?".split(),pat_to_dic(join_pat))

['why', 'does', 'iPhone', 'need', 'set', '?']

In [108]:
' '.join(subsititle("why does ?p need ?X ?".split(),pat_to_dic(join_pat)))

'why does iPhone need set ?'

In [109]:
defined_patterns = {
     "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
     "My ?X told me something": ["Talk about more about your ?X", 
                                "How do you think about your ?X ?"]
}

In [110]:
pattern_1 = pat_match("I need ?X".split(),"I need iPhone".split())

?X


In [111]:
' '.join(subsititle("Image you will get ?X soon".split(),pat_to_dic(pattern_1)))

'Image you will get iPhone soon'

In [112]:
import random

In [113]:
defined_patterns

{'I need ?X': ['Image you will get ?X soon', 'Why do you need ?X ?'],
 'My ?X told me something': ['Talk about more about your ?X',
  'How do you think about your ?X ?']}

有点理解，但是没有get到那个main idea .

In [114]:
def get_response(saying,rules):
    for key in rules:
        if pat_match(key.split(),saying.split()):
            rule = pat_match(key.split(),saying.split())
            pattern = (random.choice(rules[key]))
            return ' '.join(subsititle(pattern.split(),pat_to_dic(rule)))


In [115]:
get_response("I need iPhone",defined_patterns)

?X
?X


'Why do you need iPhone ?'

In [116]:
get_response("My mother told me something",defined_patterns)

?X
?X


'Talk about more about your mother'

In [117]:
 def is_variable_segment(pattern):
    return pattern.startswith("?*") and all( a.isalpha for a in pattern[2:])

In [118]:
is_variable_segment("?*aaa")

True

In [119]:
from collections import defaultdict

In [128]:
fail = [True, None]

def pat_match_with_seg(pattern,saying):
    if not pattern or not saying: return []

    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat,saying[0]) + pat_match_with_seg(pattern[1:],saying[1:])]
    elif is_variable_segment(pat):
        match,index = segment_match(pattern,saying)
        return [match] + pat_match_with_seg(pattern[1:],saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:],saying[1:])
    else:
        return fail

In [129]:
def segment_match(pattern, saying):
    seg_pat,rest = pattern[0],pattern[1:]
    seg_pat = seg_pat.replace('?*','?')
    
    if not rest:return (seg_pat,saying),len(saying)
    
    for i,token in enumerate(saying):
        if rest[0] == token :
            return (seg_pat,saying[:i]), i
    return (seg_pat,saying),len(saying)
            

In [130]:
segment_match("?*P is very good".split(),"My computer and My iPhone is very good".split())

(('?P', ['My', 'computer', 'and', 'My', 'iPhone']), 5)

In [437]:
pat_match_with_seg("?*P is very good and ?*X".split(),"My dog is very good and my cat is very cut".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cut'])]

In [440]:
pat_match_with_seg(" I need an  ?*X".split(),"I need an iPhone".split())

[True, None]

In [289]:
def pat_to_dict(patterns):
    return {k:' '.join(v) if isinstance(v,list) else v for k,v in patterns}

In [290]:
sub = subsititle("Why do you need ?X".split(),pat_to_dict(pat_match_with_seg('I need ?*X'.split(),'I need an iPhone'.split())))

In [291]:
' '.join(sub)

'Why do you need an iPhone'

In [269]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [451]:
rules

{'?*X hello ?*Y': ['Hi, how do you do?'],
 'I was ?*X': ['Were you really ?X ?', 'I already knew you were ?X .']}

有些细节没有想通，还没有写出来

In [520]:
def get_reponse_seg(saying,rules):
    for key in rules:
        if pat_match_with_seg(key.split(),saying.split()) and pat_match_with_seg(key.split(),saying.split())!=[True ,None]:
            rule = pat_match_with_seg(key.split(),saying.split())
            pattern_4 = random.choice(rules[key])
            print (pattern_4)
            return ' '.join(subsititle(pattern_4.split(),pat_to_dict(rule)))


In [521]:
get_reponse_seg("I was went ",rules)

Hi, how do you do?


'Hi, how do you do?'

In [522]:
pat_match_with_seg("?*X hello ?*Y".split(),"I was went".split())

[('?X', ['I', 'was', 'went'])]